In [100]:
import pandas as pd
from scipy.spatial.distance import euclidean
from process import *
from utils import *
from model import *

In [101]:
#prendo i dati relativi al dataset
device='cpu'
layer=3
adj, adj_i, features, labels, idx_train, idx_val, idx_test, num_features, num_labels, g = full_load_data_custom('cornell','splits/cornell_split_0.6_0.2_0.npz')
features = features.to(device)

# processo la matrice di adiacenza del grafo per darlo in input alla rete
adj = adj.to(device)
adj_i = adj_i.to(device)
list_mat = []
list_mat.append(features)
no_loop_mat = features
loop_mat = features

for ii in range(layer):
    no_loop_mat = torch.spmm(adj, no_loop_mat)
    loop_mat = torch.spmm(adj_i, loop_mat)
    list_mat.append(no_loop_mat)
    list_mat.append(loop_mat)

In [102]:
layer_norm=bool(1)
# Define the path to the saved model checkpoint
PATH = "trained_epochs/cornell5.pt"
GNNcornell=FSGNN(nfeat=num_features,
                nlayers=len(list_mat),
                nhidden=64,
                nclass=num_labels,
                dropout=0.5).to(device)
GNNcornell.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [103]:
#prendo l'embedding dalla rete
GNNcornell.eval()
out = GNNcornell.emb(list_mat, layer_norm)
#torch.save(out,'cornell\emb_cornell1.pt')


In [104]:
out.shape

torch.Size([183, 448])

In [105]:
#trasformo la matrice di adiacenza in grafo
G = nx.Graph(g)
adj_mat=pd.DataFrame(columns=['target','source'])
adj_mat = nx.to_pandas_edgelist(G)
adj_mat = adj_mat[adj_mat['source']!=adj_mat['target']]
adj_mat

source  target  weight
0         0      42       1
1         1      90       1
2         1      82       1
3         1      99       1
4         1     148       1
..      ...     ...     ...
275     148     154       1
276     148     167       1
277     148     178       1
278     149     167       1
279     157     160       1

[277 rows x 3 columns]

In [106]:
#definizione distanza manhattan
import numpy as np

def manhattan_distance(arr1, arr2):
    """
    Calculates the Manhattan distance between two arrays of the same length.
    """
    return np.sum(np.abs(arr1 - arr2))

In [107]:
'''#aggiungo i pesi ottenuti dall'embedding
df_emb = pd.DataFrame(out.detach().numpy(), columns=['feature_' + str(x) for x in range(448)])
adj_mat['weight'] = adj_mat.apply(lambda x:
                                    manhattan_distance(df_emb.loc[x['target']].values,
                                    df_emb.loc[x['source']].values),
                                    axis=1)
adj_mat.to_csv('cornell/cornell_weights.csv', index = False)'''

"#aggiungo i pesi ottenuti dall'embedding\ndf_emb = pd.DataFrame(out.detach().numpy(), columns=['feature_' + str(x) for x in range(448)])\nadj_mat['weight'] = adj_mat.apply(lambda x:\n                                    manhattan_distance(df_emb.loc[x['target']].values,\n                                    df_emb.loc[x['source']].values),\n                                    axis=1)\nadj_mat.to_csv('cornell/cornell_weights.csv', index = False)"

In [108]:
#aggiungo i pesi ottenuti dall'embedding
from sklearn.metrics.pairwise import cosine_similarity

df_emb = pd.DataFrame(out.detach().numpy(), columns=['feature_' + str(x) for x in range(448)])
adj_mat['weight'] = adj_mat.apply(lambda x:
                                    cosine_similarity(np.array([df_emb.iloc[x['target']]]),
                                    np.array([df_emb.iloc[x['source']]]))[0][0],
                                    axis=1)
adj_mat.to_csv('weight_epochs/cornell_cosine_weight5.csv', index = False)

In [109]:
#label della rete 
pd.DataFrame(labels.numpy(),columns=['label']).to_csv('weight_epochs/cornell_labels.csv', index = False)